In [ ]:
import numpy as np
import sqlite3
import pandas as pd

# Função para conectar ao banco de dados SQLite
def conectar_banco_de_dados():
    return sqlite3.connect('projeto_teste.db')

# Carregar o dataset
df = pd.read_csv("diabetes.csv")

# Exibir informações do DataFrame
#print(df.shape)
#print(df.head())

# Criar conexão com o banco
conn = conectar_banco_de_dados()

# Salvar DataFrame no banco SQLite
df.to_sql("diabetes", conn, if_exists="replace", index=False)

query = "SELECT Age, Glucose, Outcome FROM diabetes WHERE Glucose > 190"
df_glicose_alta = pd.read_sql(query,conn)
print(df_glicose_alta)




# Fechar a conexão
conn.close()


In [ ]:
create_table_query = """
CREATE TABLE pacientes (
    Pregnancies INT,
    Glucose INT,
    BloodPressure INT,
    SkinThickness INT,
    Insulin INT,
    BMI DECIMAL(8, 2),
    DiabetesPedigreeFunction DECIMAL(8, 2),
    Age INT,
    Outcome INT
);
"""
conn = conectar_banco_de_dados()
cursor = conn.cursor()


cursor.execute(create_table_query)

conn.commit()
conn.close()

In [14]:
insert_query = """
INSERT INTO pacientes (Pregnancies, 
                       Glucose, 
                       BloodPressure, 
                       SkinThickness, 
                       Insulin, 
                       BMI, 
                       DiabetesPedigreeFunction, 
                       Age, 
                       Outcome) 
SELECT Pregnancies, 
       Glucose, 
       BloodPressure, 
       SkinThickness, 
       Insulin, 
       BMI, 
       DiabetesPedigreeFunction, 
       Age, 
       Outcome 
FROM diabetes WHERE Age > 50;
"""

conn = conectar_banco_de_dados()
cursor = conn.cursor()


cursor.execute(insert_query)

conn.commit()
conn.close()

In [21]:
query = "SELECT * FROM pacientes"
conn = conectar_banco_de_dados()
consultar_pacientes = pd.read_sql(query,conn)
display(consultar_pacientes)

conn.close()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,perfil
0,2,197,70,45,543,30.5,0.158,53,1,None
1,8,125,96,0,0,0.0,0.232,54,1,normal
2,10,139,80,0,0,27.1,1.441,57,0,normal
3,1,189,60,23,846,30.1,0.398,59,1,None
4,5,166,72,19,175,25.8,0.587,51,1,normal
...,...,...,...,...,...,...,...,...,...,...
76,5,97,76,27,0,35.6,0.378,52,1,None
77,2,105,75,0,0,23.3,0.560,53,0,normal
78,0,123,72,0,0,36.3,0.258,52,1,None
79,6,190,92,0,0,35.5,0.278,66,1,None


In [ ]:
add_coluna_perfil = "ALTER TABLE pacientes ADD perfil VARCHAR(10)"
conn = conectar_banco_de_dados()
cursor = conn.cursor()


cursor.execute(add_coluna_perfil)

conn.commit()
conn.close()

In [20]:
alimentar_coluna_perfil = "UPDATE pacientes SET perfil = 'normal' WHERE BMI < 30 "
conn = conectar_banco_de_dados()
cursor = conn.cursor()


cursor.execute(alimentar_coluna_perfil)

conn.commit()
conn.close()

In [ ]:
conn = conectar_banco_de_dados()
cursor = conn.cursor()

cursor.execute("SELECT * FROM pacientes")

colunas = [coluna[0] for coluna in cursor.description]



novo_dataframe = pd.DataFrame.from_records(data= cursor.fetchall(),columns= colunas)


conn.close()


display(novo_dataframe)

novo_dataframe.to_csv("pacientes.csv", index=False, encoding="utf-8")

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,perfil
0,2,197,70,45,543,30.5,0.158,53,1,None
1,8,125,96,0,0,0.0,0.232,54,1,normal
2,10,139,80,0,0,27.1,1.441,57,0,normal
3,1,189,60,23,846,30.1,0.398,59,1,None
4,5,166,72,19,175,25.8,0.587,51,1,normal
...,...,...,...,...,...,...,...,...,...,...
76,5,97,76,27,0,35.6,0.378,52,1,None
77,2,105,75,0,0,23.3,0.560,53,0,normal
78,0,123,72,0,0,36.3,0.258,52,1,None
79,6,190,92,0,0,35.5,0.278,66,1,None


In [ ]:
%pip install -q ipython-sql